In [ ]:
import cv2
import mediapipe as mp
import math
import numpy
    
from ctypes import cast,POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities,IAudioEndpointVolume

devices= AudioUtilities.GetSpeakers()
interface =devices.Activate(IAudioEndpointVolume._iid_,CLSCTX_ALL,None)
volume=cast(interface,POINTER(IAudioEndpointVolume))

volume.SetMasterVolumeLevel(-65.25,None)
cap= cv2.VideoCapture(0)
mpdraw =mp.solutions.drawing_utils
mphands=mp.solutions.hands
hands=mphands.Hands()

while True:
    success,img= cap.read()
    imgRGB =cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    outcome=hands.process(imgRGB)  #to detect palm
    
 #Looping through coordinates of all the hands
    if outcome.multi_hand_landmarks:
        for handLms in outcome.multi_hand_landmarks:
            l=[]
            for id,lm in enumerate(handLms.landmark):
                h,w,c=img.shape
                cx,cy=int(lm.x*w), int(lm.y*h)
                l.append([id,cx,cy])
                
            if l:
                x1,y1=l[4][1],l[4][2]  #Coordinates of Thumb
                x2,y2 = l[8][1],l[8][2] #Cordinates of Forefinger
                cv2.circle(img,(x1,y1),15, (1,23,123), cv2.FILLED) #CRCLE around thumb tip
                cv2.circle(img,(x2,y2),15, (1,23,123), cv2.FILLED)   #cIRCLE around FORE-Finger tip        
                cv2.line(img,(x1,y1),(x2,y2), (1,23,123),4)     
                z1,z2 =(x1+x2)//2,(y1+y2)//2   #To find the distance between two fingers 
                flength=math.hypot(x2-x1,y2-y1)
                
            volrange = volume.GetVolumeRange()
            minvol=volrange[1]
            maxvol=volrange[0]
            vol=numpy.interp(flength,[50,100],[maxvol,minvol])
            volper=numpy.interp(flength,[50,100],[0,100])
            volbar=numpy.interp(flength,[50,300],[400,150])
            volume.SetMasterVolumeLevel(vol,None)
            
            cv2.putText(img,str(int(volper)),(40,450),cv2.FONT_HERSHEY_COMPLEX,5,(1,3,5),3)
            cv2.rectangle(img,(50,150),(85,400),(123,213,122),3)
            cv2.rectangle(img,(50,int(volbar)),(85,400),(0,231,23),cv2.FILLED)

    cv2.imshow("Gesture Sound Control",img)
    cv2.waitKey(1)